In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time

import warnings
warnings.filterwarnings('ignore')

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/code/latest/1_tall2wide/2_tall2wide/latest/phase_III/LPR/tall2wide


### Reformat LPR behavioral data

In [3]:
#mac       - phase III - LPR
complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/behavior/LPR/tall/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23_full_w_240_timepoints.csv'

#constance
#complete_input_file_path= '/people/kimd999/tox/phase_III/LPR/input_11PM/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23_full_w_240_timepoints.csv'

df_behav = pd.read_csv(complete_input_file_path, header = 0)
display(len(np.unique(df_behav['chemical.id'])))
df_behav = df_behav.rename({'variable': 'timepoint'}, axis=1)
display(df_behav.head())

215

,chemical.id,conc,plate.id,well,timepoint,value
0,234,0.0,14389,H01,t0,17.223000
1,234,0.0,14389,H02,t0,12.876000
2,234,0.0,14389,H03,t0,24.156000
3,234,0.0,14389,H04,t0,31.421999
4,234,0.0,14389,H05,t0,36.917999


In [4]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'timepoint', 'value']
df_select = df_behav.loc[:,columns_to_keep]
df_select.head()

,chemical.id,conc,plate.id,well,timepoint,value
0,234,0.0,14389,H01,t0,17.223000
1,234,0.0,14389,H02,t0,12.876000
2,234,0.0,14389,H03,t0,24.156000
3,234,0.0,14389,H04,t0,31.421999
4,234,0.0,14389,H05,t0,36.917999


In [5]:
display("chemical_id_count:"+str(len(np.unique(df_select['chemical.id']))))

'chemical_id_count:215'

In [6]:
nan = df_select[df_select['value'].isna()]
display(nan.head())

# [phase III] there is no nan in 'chemical.id', 'conc', 'plate.id', 'well', 'variable'

# don't drop na now for easier proceesing for now

### Transpose time points

In [14]:
start_time = time.time()

df_reformatted = pd.DataFrame()
    
len_group_timepoint = []
    
full_devel = "full"
#full_devel = "devel"


chemical_id_from_here = np.unique(df_behav['chemical.id'])

if (full_devel == "devel"):
    #choose_this_number = min(len(chemical_id_from_here), 1)
    choose_this_number = min(len(chemical_id_from_here), 5)
    randomly_chosen = random.sample(set(chemical_id_from_here), choose_this_number)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

    
for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    df_per_chemical = df_select.loc[df_behav['chemical.id'] == chemical_index,:]
    print("len(df_per_chemical):" + str(len(df_per_chemical)))
    
    
    # Append chemical_plate_well as a unique identifier
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    #output_filename = str(chemical_index) + ".csv"
    #behav_data_chemical.to_csv(output_filename,index=False)
    
    for cpw in np.unique(df_per_chemical.chemical_plate_well):
        df_per_cpw = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]

        df_per_cpw_grouped = df_per_cpw.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in df_per_cpw_grouped:
            
            #display("group.timepoint", group.timepoint.head())
            #len_group_timepoint.append(str(len(group.timepoint)))
            len_group_timepoint.append(str(len(np.unique(group.timepoint))))
            df_to_concat = pd.DataFrame(
                    {
                    'chemical.id': np.unique(df_per_cpw['chemical.id']),
                    'plate.id': np.unique(df_per_cpw['plate.id']),
                    'well': np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc': np.unique(df_per_cpw['conc'])
                    })
            #display("before concat:", df_to_add.head())
            
            # Append additonal columns corresponding to time points
            for time_point in np.arange(len(group.timepoint)):
                timepoint = 't'+ str(time_point)
                
                #print ("pd.DataFrame({end_point: per_cpw.value[per_cpw.timepoint == end_point]}):"\
                #   +str(pd.DataFrame({end_point: per_cpw.value[per_cpw.timepoint == end_point]})))
                #print ("pd.DataFrame({end_point: per_cpw.value[per_cpw.timepoint == end_point].values}):"\
                #   +str(pd.DataFrame({end_point: per_cpw.value[per_cpw.timepoint == end_point].values})))
                df_to_concat = pd.concat([df_to_concat, pd.DataFrame({timepoint: df_per_cpw.value[df_per_cpw.timepoint == timepoint].values})],axis = 1)
            #display("after concat:", temp.head())

            df_reformatted = pd.concat([df_reformatted, df_to_concat])
            
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds.\n"

f = open("time_took_for_tall2wide.txt", "w")
f.write(time_took)
f.close()

print ("Done, it took:"+str(time_took)) 
# took 1~3 minutes for 1 chemical
# took 5 hrs in constance for 215 chemicals

chemical_index:4417
len(df_per_chemical):40320
1 chemicals processed out of 5
chemical_index:4424
len(df_per_chemical):40320
2 chemicals processed out of 5
chemical_index:2715
len(df_per_chemical):69120
3 chemicals processed out of 5
chemical_index:4407
len(df_per_chemical):40320
4 chemicals processed out of 5
chemical_index:4387
len(df_per_chemical):40320
5 chemicals processed out of 5
Done, it took:221.2 seconds.



In [ ]:
# ds = pd.Series({"Column": len_group_variable})
# plt.figure(figsize=(8,4))
# sns.countplot(x="Column", data=ds)
# plt.show()

In [10]:
display(df_reformatted)

,chemical.id,plate.id,well,chemical_plate_well,conc,t0,t1,t2,t3,t4,...,t230,t231,t232,t233,t234,t235,t236,t237,t238,t239
0,1607,14350,A01,1607_14350_A01,67.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1607,14350,A02,1607_14350_A02,67.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1607,14350,A03,1607_14350_A03,67.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1607,14350,A04,1607_14350_A04,67.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1607,14350,A05,1607_14350_A05,67.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1898,12535,H08,1898_12535_H08,0.0,0.000,0.000,0.000,0.000,0.000,...,15.943,15.983,20.287999,8.875,23.472,0.712000,0.000,0.000000,0.000,0.000000
0,1898,12535,H09,1898_12535_H09,0.0,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000000,0.000,0.000,0.000000,4.444,3.262000,0.000,0.000000
0,1898,12535,H10,1898_12535_H10,0.0,2.696,7.485,5.444,7.212,3.157,...,0.000,0.000,0.000000,0.000,3.909,2.518000,0.000,0.000000,0.000,5.910000
0,1898,12535,H11,1898_12535_H11,0.0,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000000,0.000,0.000,0.000000,0.000,0.000000,0.000,0.000000


In [15]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_" + str(full_devel) + ".csv"
print ("reformatted_data_filename:" + str(reformatted_data_filename))
df_reformatted.to_csv(reformatted_data_filename,index=False)

reformatted_data_filename:/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/behavior/LPR/tall/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23_full_w_240_timepoints_wide_devel.csv


In [ ]:
#display(len(np.unique(reformatted_w_240_endpoints['chemical.id'])))
#display(len(np.unique(reformatted_w_non_240_endpoints['chemical.id'])))

In [ ]:
'''reformatted_w_non_240_endpoints_155 = reformatted_w_non_240_endpoints[reformatted_w_non_240_endpoints['chemical.id']==155]

display(reformatted_w_non_240_endpoints_155.head())
display(reformatted_w_non_240_endpoints_155.tail())

output_filename = str(complete_input_file_path[:-4]) + "_155_chemical_id.csv"
print ("output_filename:"+str(output_filename))
reformatted_w_non_240_endpoints_155.to_csv(output_filename, index=False)'''

In [ ]:
'''reformatted_w_non_240_endpoints_163 = reformatted_w_non_240_endpoints[reformatted_w_non_240_endpoints['chemical.id']==163]

display(reformatted_w_non_240_endpoints_163.head())
display(reformatted_w_non_240_endpoints_163.tail())

output_filename = str(complete_input_file_path[:-4]) + "_163_chemical_id.csv"
print ("output_filename:"+str(output_filename))
reformatted_w_non_240_endpoints_163.to_csv(output_filename, index=False)'''